# RTM Example with dynamic scheduling showing wavefield separation
We will perform RTM using the following steps:
1. Read the 10m resampled models output from the FWI notebook
2. Visualize the model
3. Build a small local compute cluster (2 workers)
4. Create list of shot locations 
5. Define the `timemute!`, `migrateshot`, and `stack` functions
6. Run the migration and write individual shot images to disk
7. Stack the individual shot images 
8. Perform a little post migration filtering
9. Visualize Results

#### Note on runtime
This notebook takes approximately 20 minutes to run for 100 shots with two workers on an Intel 8168.

`lscpu` CPU information: `Intel(R) Xeon(R) Platinum 8168 CPU @ 2.70GHz`

## Add required packages

In [ ]:
using Distributed, PyPlot, Jets, JetPack, JetPackDSP, Printf

## Record time for start of notebook

In [ ]:
time_beg = time()

## 1. Read the 10m resampled models output from the FWI notebook

In [ ]:
file1 = "../50_fwi/marmousi_resampled_10m_349x1701_vtrue.bin"

nz,nx = 349,1701
dz,dx = 10.0,10.0

v1_orig = read!(file1, Array{Float32}(undef,nz,nx));

####  Apply a 7x7 rectangular mix smoother
We perform the RTM migration in a slightly smoothed model.

In [ ]:
ns = 21
P = JopPad(JetSpace(Float32,nz,nx), -ns:nz+ns, -ns:nx+ns, extend=true)
M = JopMix(range(P), (7,7))
R = JopPad(JetSpace(Float32,nz,nx), -ns:nz+ns, -ns:nx+ns, extend=false)

s1 = R' ∘ M ∘ P * (1 ./ v1_orig)

v1 = 1 ./(s1);

## 2. Visualize the model

In [ ]:
vmin,vmax = extrema(v1_orig)
@show vmin,vmax;
figure(figsize=(8,3.5)); clf()

imshow(v1,aspect="auto",cmap="jet");
colorbar(orientation="vertical");clim(vmin,vmax);
title("True Velocity");
tight_layout()

## 3. Build a small local compute cluster (2 workers) 

#### Setup OMP environment variables for the cluster

In the distributed compute case the workers that we add would be on different hardware, and we might add tens of workers in 2D and hundreds in 3D. Here we run on a single machine with only 2 workers, and so we need to be careful with details related to high performance computing. If we did not specify thread affinity, the two workers would compete for the same physical cores and the modeling would be *incredibly* slow.

We spin up the small 2-worker cluster by calling `addprocs(2)`, and because we set the environment variable `ENV["OMP_DISPLAY_ENV"] = "true"` we will see the OMP environment printed out on each worker. In that output (below) we can verify that half of the total threads (44/2 = 22) are assigned to each socket on this 2 socket system. You can obtain more details about the hardware with the shell command `lscpu`.

We set four environment variables related to OpenMP:
* `OMP_DISPLAY_ENV` prints out the OpenMP environment on each worker
* `OMP_PROC_BIND` specifies that threads should be bound to physical cores
* `OMP_NUM_THREADS` specifies the number of threads per workers is 1/2 the number of physical cores
* `GOMP_CPU_AFFINITY` specifies which physical cores the threads run on for each worker

If you run the shell command `top` during execution, you will see 3 julia processes: the main process and two workers. The two workers should generally have about 50% of the system, and `load average` should tend towards the physical number of cores.

In [ ]:
nthread = Sys.CPU_THREADS
ENV["OMP_DISPLAY_ENV"] = "true"
ENV["OMP_PROC_BIND"] = "close"
ENV["OMP_NUM_THREADS"] = "$(div(nthread,2))" 
addprocs(2)
@show workers()
for k in 1:nworkers()
    place1 = (k - 1) * div(nthread,nworkers())
    place2 = (k + 0) * div(nthread,nworkers()) - 1
    @show place1, place2, nthread
    @spawnat workers()[k] ENV["GOMP_CPU_AFFINITY"] = "$(place1)-$(place2)";
end

In [ ]:
@everywhere using DistributedArrays, DistributedJets, DistributedOperations, Jets, JetPack, WaveFD, JetPackWaveFD, Random, LinearAlgebra, Schedulers

## 4. Create list of shot locations 
We use 100 shot locations, many times than our FWI example, and run at significantly higher frequency. 

In [ ]:
nshots = 100
sx = round.(Int,collect(range(0,stop=(nx-1)*dx,length=nshots)))
@show nshots;
@show sx;

## 5. Define the `timemute!`, `migrateshot`, and `stack` functions
* `timemute!` mutes data to remove the direct arrival and refraction
* `migrateshot` runs the migrations for each shot and writes image and illumination files to scratch disk
* `stack` reads in shots from disk and stacks them

**TODO:** convert the IO to use CloudSeis as proxy for cloud storage

#### Note on scratch space for temporary files
When dealing with serialized nonlinear wavefields as in this example, we need to specify the location where scratch files will be written.

You may need to change this to point to a temporary directory available on your system.

In [ ]:
@everywhere scratch = "/mnt/scratch"
@assert isdir(scratch)

#### Global variables for number of samples and sample rate

In [ ]:
@everywhere begin
    ntrec = 3001
    dtrec = 0.002
    dtmod = 0.001
end

#### Build the `timemute!` function 

In [ ]:
@everywhere function timemute!(F, d, watervel, tmute)
    for i = 1:length(state(F, :rx)) 
        rx = state(F, :rx)
        rz = state(F, :rz)
        sx = state(F, :sx)
        sz = state(F, :sz)
        dist = sqrt((sx[1] - rx[i])^2 + (sz[1] - rz[i])^2)
        time = dist / watervel
        tbeg = 1
        tend = round(Int, (time + tmute) / state(F,:dtrec))
        tend = clamp(tend,1,size(d,1))
        d[tbeg:tend,i] .= 0
    end
    nothing
end

#### Build the `migrateshot` function

In [ ]:
# notice we model to significantly shorter time than for FWI
@everywhere function migrateshot(isrc,nz,nx,dz,dx,_vtrue,_v,sx)
    @info "migrating shot $(isrc) on $(gethostname()) with id $(myid()) ... IC RTM"
    F1 = JopNlProp2DAcoIsoDenQ_DEO2_FDTD(;
        b = ones(Float32,nz,nx),
        nthreads = div(Sys.CPU_THREADS,2),
        ntrec = ntrec,
        dtrec = dtrec,
        dtmod = dtmod,
        dz = dz,
        dx = dx,
        wavelet = WaveletCausalRicker(f=10.0),
        sx = sx[isrc],
        sz = dz,
        rx = dx*[0:1:nx-1;],
        rz = 2*dz*ones(length(0:1:nx-1)),
        nbz_cache = nz,
        nbx_cache = 16,
        comptype = UInt32,
        imgcondition = "standard",
        srcfieldfile = joinpath(scratch, "field-$isrc-$(randstring()).bin"),
        reportinterval=12001)

    @info "migrating shot $(isrc) on $(gethostname()) with id $(myid()) ... IC standard"
    F2 = JopNlProp2DAcoIsoDenQ_DEO2_FDTD(;
        b = ones(Float32,nz,nx),
        nthreads = div(Sys.CPU_THREADS,2),
        ntrec = ntrec,
        dtrec = dtrec,
        dtmod = dtmod,
        dz = dz,
        dx = dx,
        wavelet = WaveletCausalRicker(f=10.0),
        sx = sx[isrc],
        sz = dz,
        rx = dx*[0:1:nx-1;],
        rz = 2*dz*ones(length(0:1:nx-1)),
        nbz_cache = nz,
        nbx_cache = 16,
        comptype = UInt32,
        imgcondition = "RTM",
        srcfieldfile = joinpath(scratch, "field-$isrc-$(randstring()).bin"),
        reportinterval=12001)

    d = F1*localpart(_vtrue) #here we model the data usually you would just read the data
    timemute!(F1,d,1500,2/16) #mute out the direct and diving waves
    J1 = jacobian!(F1, localpart(_v))
    J2 = jacobian!(F2, localpart(_v))
    illum = srcillum(J1)
    m1 = J1'*d
    m2 = J2'*d
    close(F) #delete scratch files that we don't need anymore
    @info "writing image and illumination for shot $(isrc)"
    write(joinpath(scratch,"image1_$(isrc).bin"),m1)
    write(joinpath(scratch,"image2_$(isrc).bin"),m2)
    write(joinpath(scratch,"illum_$(isrc).bin"),illum)
    @info "done migrating shot $(isrc) on $(gethostname()) with id $(myid())..."
end

#### Build the `stack` function

In [ ]:
function stack(shots,nz,nx)
   img1 = zeros(Float32,nz,nx)
   img2 = zeros(Float32,nz,nx)
   ill = zeros(Float32,nz,nx)
   for isrc in shots
      img1 += read!(joinpath(scratch,"image1_$(isrc).bin"), Array{Float32}(undef,nz,nx));
      img2 += read!(joinpath(scratch,"image2_$(isrc).bin"), Array{Float32}(undef,nz,nx));
      ill += read!(joinpath(scratch,"illum_$(isrc).bin"), Array{Float32}(undef,nz,nx));
      rm(joinpath(scratch,"image1_$(isrc).bin"))
      rm(joinpath(scratch,"image2_$(isrc).bin"))
      rm(joinpath(scratch,"illum_$(isrc).bin"))
   end
   return img1,img2,ill
end

## 6. Run the migration and write individual shot images to disk

We use epmap to schedule the work for the migration.

In [ ]:
#broadcast the models to the workers
_v1 = bcast(v1);

In [ ]:
t1 = @elapsed begin
    epmap(i->migrateshot(i, nz, nx, dz, dx, _v1, _v1, sx), 1:nshots)
end
@show t1;

In [ ]:
@printf("Time for migrating %.2f minutes\n", t1 / 60)

## 7. Stack the individual shot images 

In [ ]:
shots = collect(1:nshots)
m1, m2, illum1 = stack(shots,nz,nx);

## 8. Perform a little post migration filtering

#### Laplacian filter to remove backscattered noise

In [ ]:
L = JopHighpass(JetSpace(Float32,nz,nx))

#### Apply low cut filter, illumination compensation, and gain

In [ ]:
g = ([0:(nz-1);]*dz).^2 * ones(1,nx);

m1 = g .* m1 ./ (illum1 .+ 1e-8 * maximum(abs, illum1));
m2 = g .* m2 ./ (illum1 .+ 1e-8 * maximum(abs, illum1));

img1 = g .* (L * m1) ./ (illum1 .+ 1e-8 * maximum(abs, illum1));
img2 = g .* (L * m2) ./ (illum1 .+ 1e-8 * maximum(abs, illum1));

@show extrema(img1)

#### Apply water bottom mute

In [ ]:
img1[v1.==1500.0] .= 0;
img2[v1.==1500.0] .= 0;

## 9. Visualize Results

In [ ]:
mrms1 = 2.5 * sqrt(norm(m1)^2 / length(img1))
mrms2 = 2.5 * sqrt(norm(m2)^2 / length(img1))
irms1 = 2.5 * sqrt(norm(img1)^2 / length(img1))
irms2 = 2.5 * sqrt(norm(img2)^2 / length(img1))

figure(figsize=(8,14)); clf()

subplot(2,2,1); imshow(m1,aspect="auto",cmap="gray");
colorbar(orientation="vertical");clim(-mrms1,+mrms1);
title("Raw Migration with Standard Imaging Condition")

subplot(2,2,1); imshow(m2,aspect="auto",cmap="gray");
colorbar(orientation="vertical");clim(-mrms2,+mrms2);
title("Raw Migration with FWI Imaging Condition")

subplot(2,2,1); imshow(img1,aspect="auto",cmap="gray");
colorbar(orientation="vertical");clim(-irms1,+irms1);
title("Processed Migration with Standard Imaging Condition")

subplot(2,2,1); imshow(img2,aspect="auto",cmap="gray");
colorbar(orientation="vertical");clim(-irms2,+irms2);
title("Processed Migration with FWI Imaging Condition")

tight_layout()

## Remove workers

In [ ]:
rmprocs(workers())

In [ ]:
time_end = time()
@sprintf("Time to run notebook; %.2f minutes\n", (time_end - time_beg) / 60)